In [1]:
# @title
from IPython.display import HTML
shell = get_ipython()

def adjust_font_size():
  display(HTML('''<style>
    body {
      font-size: 18px;
    }
  '''))

if adjust_font_size not in shell.events.callbacks['pre_execute']:
  shell.events.register('pre_execute', adjust_font_size)

# Building simple CNN in JAX/Flax for classifying MNIST digits

In [2]:
import jax
import jax.numpy as jnp
import flax.linen as nn
from flax.training import train_state  # Useful dataclass to keep train state
import optax

## Dataset pipline

In [3]:
import tensorflow as tf

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values.
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert the labels to one-hot encoding.
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

def create_dataset(training: bool = True):
  x, y = (x_train, y_train) if training else (x_test, y_test)
  dataset = (
      tf.data.Dataset
      .from_tensor_slices((x, y))
      .shuffle(buffer_size=5000)
      .batch(64)
      .repeat()
      .prefetch(tf.data.AUTOTUNE)
      .as_numpy_iterator())
  return dataset

def get_batch(dataset):
  images, labels = next(dataset)
  images, labels = jnp.array(images), jnp.array(labels, dtype=jnp.int32)
  images = jnp.expand_dims(images, axis=-1)
  return images, labels


# Create a tf.data pipeline for the train and test set.
train_dataset = create_dataset()
test_dataset = create_dataset(training = False)

11490434/11490434 [==============================] - 0s 0us/step


### Let's test fetching a batch...

In [4]:
images_batch, labels_batch = get_batch(train_dataset)
single_image, single_label = images_batch[0], labels_batch[0]

In [5]:
single_label

Array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

## Modeling

In [6]:
import flax.linen as nn

class Model(nn.Module):
  def setup(self):
    # 28x28x1 -> 14x14x16
    self.conv1 =  nn.Conv(16, kernel_size=(3, 3), strides=(2, 2))

    # 14x14x16 -> 7x7x8
    self.conv2 = nn.Conv(8, kernel_size=(3, 3), strides=(2, 2))

    # 7x7x8 -> 4x4x4
    self.conv3 = nn.Conv(4, kernel_size=(3, 3), strides=(2, 2))

    # 4x4x4 -> 2x2x1
    self.conv4 = nn.Conv(1, kernel_size=(3, 3), strides=(2, 2))

    self.final_layer = nn.Dense(10)

  def __call__(self, x):
    x = nn.relu(self.conv1(x))
    x = nn.relu(self.conv2(x))
    x = nn.relu(self.conv3(x))
    x = nn.relu(self.conv4(x))

    # flatten
    batch_size = x.shape[0]
    x = x.reshape(batch_size, -1)

    x = self.final_layer(x)
    return x

## How to implement training Loop in Flax



### Since, JAX functions are pure, you need to maintain any state outside the actual function.

### Even the model weights need to be handled by your code and passed explicitly whenever you do forward pass or backward pass.

In [7]:
# Initialize the model using `model.init`, the function returns the model params.
model = Model()
params = model.init(jax.random.PRNGKey(99), images_batch)
params = params["params"]

In [8]:
# @title
params

{'conv1': {'kernel': Array([[[[ 0.10318059, -0.36546168,  0.10348582, -0.18727979,
            -0.2989344 , -0.06120202, -0.30305564, -0.06325838,
            -0.47007623,  0.00182239,  0.08036909, -0.28868243,
            -0.1755667 , -0.46929264,  0.22673424,  0.1213199 ]],
  
          [[-0.00159304, -0.06865791, -0.48096153,  0.5176608 ,
            -0.24610692,  0.12146369,  0.31982005,  0.19794294,
            -0.4636128 ,  0.13819885, -0.17930195,  0.1044026 ,
            -0.54910463,  0.4998827 , -0.33488676,  0.5588218 ]],
  
          [[-0.01486812, -0.27757135,  0.65702665, -0.28058687,
            -0.37632743,  0.06171224,  0.63409156,  0.1940167 ,
             0.55188036,  0.35217246, -0.39919505, -0.42158285,
            -0.2778281 , -0.01271519, -0.72809803, -0.5164525 ]]],
  
  
         [[[ 0.05771004, -0.03288956, -0.14206718, -0.3323581 ,
            -0.17923449,  0.18122466,  0.72265977,  0.53838944,
            -0.19212681,  0.2795688 , -0.23983324,  0.0612007 ,
  

# How to do forward pass in JAX?

### You call \`model.apply\` and explicitly pass the model weights and the current minibatch.

In [9]:
logits = model.apply({"params": params}, images_batch)

### Forward pass returns logits which are un-normalized probability over 10 classes.

In [10]:
logits.shape

(64, 10)

# Implementing the backward pass and training loop

In [11]:
# Step 1: create train state

In [12]:
state = train_state.TrainState.create(
    apply_fn=model.apply,  # forward pass func
    params=params,   # model weights
    tx=optax.adam(learning_rate=0.001)  # optimizer func
)

In [13]:
# Step 2: write forward and backward pass func

In [14]:
def forward_pass(params, state, batch):
  inputs, labels = batch

  # call forward pass function.
  logits = state.apply_fn({"params": params}, inputs)

  # compute loss
  loss = optax.softmax_cross_entropy(logits=logits, labels=labels)
  loss = loss.mean()
  return loss, logits

In [15]:
def backward_pass(state, batch):
  grad_fn = jax.value_and_grad(forward_pass, argnums=(0), has_aux=True)  # differentiate wrt 0th pos argument.
  (loss, _) , grads = grad_fn(state.params, state, batch)
  state = state.apply_gradients(grads=grads)
  return state, loss

In [16]:
# Step 3: put together the training loop

In [17]:
def train_step(state, batch):
  return backward_pass(state, batch)

In [20]:
def eval_step(state, batch):
  _, labels = batch

  # do forward pass.
  loss, logits = forward_pass(state.params, state, batch)

  # calculate accuracy of prediction over eval dataset.
  pred_labels = jnp.argmax(nn.softmax(logits), axis=-1)

  labels = jnp.argmax(labels, axis=-1)
  acc = jnp.mean(labels == pred_labels)

  return loss, acc

In [21]:
# Train for 1000 steps
for step in range(1000):
  train_batch = get_batch(train_dataset)

  state, loss = train_step(state, train_batch)
  if step%100==0:
    eval_batch = get_batch(test_dataset)
    eval_loss, eval_acc = eval_step(state, eval_batch)
    print("train_loss:", loss, "\teval loss:", eval_loss, "\teval acc:", eval_acc*100)

train_loss: 0.7821827 	eval loss: 0.71475947 	eval acc: 70.3125
train_loss: 0.91101336 	eval loss: 1.0669804 	eval acc: 75.0
train_loss: 0.7025645 	eval loss: 0.86087346 	eval acc: 75.0
train_loss: 0.70819485 	eval loss: 0.8867424 	eval acc: 75.0
train_loss: 0.99967396 	eval loss: 0.6607022 	eval acc: 76.5625
train_loss: 0.78139436 	eval loss: 0.6273515 	eval acc: 82.8125
train_loss: 0.59428424 	eval loss: 0.627124 	eval acc: 78.125
train_loss: 0.75524557 	eval loss: 0.48956695 	eval acc: 81.25
train_loss: 0.6262825 	eval loss: 0.49753153 	eval acc: 84.375
train_loss: 0.60644317 	eval loss: 0.5877735 	eval acc: 79.6875
